# Analyse all house of commons speeches since 1970

[Part 1: Get a list of MPs and their affiliations](MP_speeches-Part1.ipynb)

[Part 2: Download all speeches belonging to MPs in list](MP_speeches-Part2.ipynb)

[Part 3: Train bigram and trigram models and use them on all speeches](MP_speeches-Part3.ipynb)

## Part 4: Train an LDA topic model and process all speeches with it

[Part 5: Analyse the results of the LDA model](MP_speeches-Part5.ipynb)

In [2]:
import pandas as pd
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.word2vec import LineSentence
import gensim
import os
import warnings
warnings.filterwarnings('ignore')

from config import INTERMEDIATE_DIRECTORY

#### Learn the dictionary (list of words) for the whole corpus

In [41]:
%%time
# this is a bit time consuming - make the if statement True
# if you want to relearn the dictionary.
trigram_speeches_filepath = os.path.join(INTERMEDIATE_DIRECTORY, 'trigram_transformed_speeches_all.txt')
trigram_dictionary_filepath = os.path.join(INTERMEDIATE_DIRECTORY, 'trigram_dict_all.dict')
if False:
    trigram_speeches = LineSentence(trigram_speeches_filepath)

    # learn the dictionary by iterating over all of the speeches
    trigram_dictionary = Dictionary(trigram_speeches)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
else: 
    # load the finished dictionary from disk
    trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 45 ms


/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:22:47.971573. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


#### Turn speeches into bag-of-words representations

In [42]:
def trigram_bow_generator(filepath):
    """
    generator function to read speeches from a file
    and yield a bag-of-words representation
    """
    
    for speech in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(speech)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:22:50.589193. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [61]:
%%time
trigram_bow_filepath = os.path.join(INTERMEDIATE_DIRECTORY, 'trigram_bow_corpus_all.mm')
# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if False:
    # generate bag-of-words representations for
    # all speches and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_speeches_filepath))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


#### Train the LDA topic model on the speech corpus
If you want the model to use more or fewer topics, then change num_topics below. However, bare in mind that you will have to relabel all the topics yourself if you do this.

In [3]:
%%time
## Train the LDA topic model using Gensim
from gensim.models.ldamulticore import LdaMulticore

# this is a bit time consuming (takes about 45 mins)- make the if statement True
# if you want to train the LDA model yourself.
lda_model_filepath = os.path.join(INTERMEDIATE_DIRECTORY, 'lda_model_all')
if False:
    # load the finished bag-of-words corpus from disk
    trigram_bow_corpus = MmCorpus(trigram_bow_filepath)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=100,
                           id2word=trigram_dictionary,
                           workers=4)
    
    lda.save(lda_model_filepath)
else:
    # load the finished LDA model from disk
    lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 312 ms, sys: 60 ms, total: 372 ms
Wall time: 762 ms


#### Let's explore all the topics in the LDA model we just created

In [10]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format('term', 'frequency'))

    for term, frequency in lda.show_topic(topic_number, topn=topn):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [16]:
explore_topic(0, topn=10)

term                 frequency
business             0.101
small_business       0.018
government           0.015
regulation           0.013
cost                 0.013
work                 0.007
sector               0.007
industry             0.007
’s                   0.007
small                0.007


Topic 0 seems to be about business, judging by the top 10 terms. This method of visualising topics is a bit inconvenient thugh and since we have 100 topics to label, let's use pyLDAvis instead.

#### Visualise the LDA model using pyLDAvis
pyLDAvis is a smart visualisation app that allows us to navigate through all the topics in the model

In [17]:
%%time
import pickle
import pyLDAvis.gensim
from gensim.corpora import  MmCorpus
import pyLDAvis
import os

ldavis_pickle_path = os.path.join(INTERMEDIATE_DIRECTORY, "pyldavis.p")
# Change to True if you want to recalculate the visualisation (takes about 40 mins)
if False:
    trigram_bow_corpus = MmCorpus(trigram_bow_filepath)
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                                  trigram_dictionary, sort_topics=False)
    pickle.dump(LDAvis_prepared, open(ldavis_pickle_path, "wb"))
else:
    LDAvis_prepared = pickle.load(open(ldavis_pickle_path, "rb"))

CPU times: user 336 ms, sys: 20 ms, total: 356 ms
Wall time: 503 ms


Subtract 1 from topic numbers in visualisation to get pandas index

In [ ]:
pyLDAvis.display(LDAvis_prepared)

In [44]:
%%writefile topic_names.py
# Dictionary of topic names
topic_names_100 = {
    0: "business",
    3: "immigration",
    4: "counter terrorism",
    5: "syria",
    6: "private housing",
    7: "banking",
    9: "tribunal",
    18: "bbc",
    19: "police force",
    20: "parliamentary terms",
    21: "secretary of state terms",
    23: "local authority",
    25: "domestic violence",
    26: "airport and rail expansion",
    27: "scotland",
    29: "parliamentary terms+",
    30: "single market (?)",
    32: "drugs and alcohol",
    33: "middle east",
    35: "care quality commission",
    36: "speaker of the house",
    39: "nhs",
    41: "farming",
    42: "law",
    43: "development & climate change",
    44: "fishing industry",
    45: "inquiries & reports",
    46: "northern ireland",
    47: "construction",
    49: "animal welfare",
    60: "fraud terminology",
    62: "legislation",
    63: "bill terminology",
    64: "regional stuff",
    65: "elections",
    68: "local services",
    69: "energy",
    70: "welfare reforms",
    71: "european union",
    72: "education",
    73: "money-related terms",
    74: "pensioner income",
    76: "child poverty",
    78: "sports & culture",
    79: "investment",
    84: "armed forces",
    85: "economy",
    86: "house of lords",
    87: "employee's rights",
    92: "nuclear weapons",
    95: "parliamentary terms++",
    99: "child care"
}

def topic_dict(topic_number):
    """
    return name of topic where identified
    """
    
    try:
        return topic_names_100[topic_number]
    except KeyError:
        return topic_number

Writing topic_names.py


/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:30:59.314511. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


#### Load previously computed bigram and trigram models

In [29]:
%%time
if True:
    # Load the bigram and trigram models so we can apply this to any new text
    bigram_model_filepath = os.path.join(INTERMEDIATE_DIRECTORY, 'bigram_model_all')
    trigram_model_filepath = os.path.join(INTERMEDIATE_DIRECTORY, 'trigram_model_all')
    # load the bigram model from disk
    bigram_model = gensim.models.Phrases.load(bigram_model_filepath)
    # load the trigram model from disk
    trigram_model = gensim.models.Phrases.load(trigram_model_filepath)
    # Phraser class is much faster so use this instead of Phrase
    bigram_phraser = gensim.models.phrases.Phraser(bigram_model)
    trigram_phraser = gensim.models.phrases.Phraser(trigram_model)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:14:27.357676. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


CPU times: user 1min 6s, sys: 1.61 s, total: 1min 7s
Wall time: 1min 7s


#### ...and the helper functions

In [24]:
# %load helper_functions.py
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_speech(filename):
    """
    generator function to read in speeches from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for speech in f:
            yield speech.replace('\\n', '\n')

def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse speeches,
    lemmatize the text, and yield sentences
    """
    
    for parsed_speech in nlp.pipe(line_speech(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_speech.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:11:07.906274. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [25]:
if True:
    # Load english language model from spacy
    import spacy
    nlp = spacy.load("en")

def clean_text(speech_text):
    """
    Remove stop words, lemmatize and split into tokens using the trigram parser
    and return a bag-of-words representation
    """
    
    # parse the review text with spaCy
    parsed_speech = nlp(speech_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_speech = [token.lemma_ for token in parsed_speech
                      if not punct_space(token)]
    
    # apply the bigram and trigram phrase models
    bigram_speech = bigram_phraser[unigram_speech]
    trigram_speech = trigram_phraser[bigram_speech]
    
    # remove any remaining stopwords
    trigram_speech = [term for term in trigram_speech
                      if not term in spacy.en.language_data.STOP_WORDS]
    
    return trigram_speech

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:11:14.045148. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [39]:
def lda_description(speech_text):
    """
    accept the original text of a speech and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a series containing all the topics and their probabilities in the LDA representation
    """
    
    # Get clean representation of text
    trigram_speech = clean_text(speech_text)

     # create a bag-of-words representation
    speech_bow = trigram_dictionary.doc2bow(trigram_speech)
    
    # create an LDA representation
    speech_lda = lda[speech_bow]
    
    topic_dict = dict(zip(range(100), [0.0]*100))
    
    for topic in speech_lda:
        topic_dict[topic[0]] = topic[1]
    topic_dict["n_words"] = len(trigram_speech)
    return pd.Series(topic_dict)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 07:22:10.743816. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [1]:
import pandas as pd
import numpy as np
# Read in details of MPs
mps = pd.read_hdf("list_of_mps.h5", "mps")

#### Apply LDA model to all speeches in the speeches dataframe and save to disk

In [2]:
%%time
# This takes a while (~3h) so use cached version if available
# Change to True if you want to recalculate the LDA topics for the speeches
if False:
    import dask.dataframe as dd
    import dask.threaded
    from dask.diagnostics import ProgressBar
    pbar = ProgressBar()
    pbar.register()
    
    # You should probably tweak this line to point to all speech dataframes
    # I ran this section twice using both sets of raw speeches and saved them to two sections of the processed hdf5 file
    speeches = pd.read_hdf("raw_speeches.h5", "speeches_0")
    speeches = dd.from_pandas(speeches, npartitions=8).map_partitions(lambda x: pd.concat([x, x.body.apply(lda_description, 1)], axis=1)).compute(get=dask.threaded.get)
    # And this one to save to the right hdf file. Remember to change it if you don't want to overwrite the file or section.
    speeches.to_hdf("/media/Stuff/processed_speeches_new.h5", "speeches_0")
else:
    try:
        del speeches
    except NameError:
            pass
    speeches = pd.read_hdf("/media/Stuff/processed_speeches_new.h5", "speeches_0").drop("body", axis=1)\
        .append([pd.read_hdf("/media/Stuff/processed_speeches_new.h5", "speeches_1").drop("body", axis=1)], ignore_index=True)

CPU times: user 6.82 s, sys: 15.7 s, total: 22.6 s
Wall time: 32.4 s


In [3]:
speeches["date"] = pd.to_datetime(speeches["date"])
speeches["mp_id"] = pd.to_numeric(speeches["mp_id"])
speeches["section_id"] = pd.to_numeric(speeches["section_id"])

In [4]:
speeches.head()

,date,debate_title,mp_id,mp_name,n_words,section_id,0,1,2,3,...,90,91,92,93,94,95,96,97,98,99
0,1983-01-24,Oral Answers to Questions &#8212; Trade: Video...,10574,Jack Straw,27.0,16624762,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1983-01-24,Oral Answers to Questions &#8212; Trade: Merch...,21914,James Johnson,36.0,16624762,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1983-01-24,Oral Answers to Questions &#8212; Trade: Adver...,21960,John Fraser,39.0,16624762,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1983-01-24,Oral Answers to Questions &#8212; Trade: Packa...,21960,John Fraser,13.0,16624762,0.251425,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1983-01-24,Oral Answers to Questions &#8212; Trade: Video...,21960,John Fraser,23.0,16624762,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
